# K-means online

En este notebook se desmuestra como utilizar el algoritmo k-means en línea.

Utilizaremos el algoritmo "común" usando la librería de sklearn para utilizarlo como base de comparación.

Describiremos dos algoritmos de k-means en línea:
* Usando solamente numpy
* Usando sklearn pero usando mini batches de datos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#--para crear un dataset artificial
from sklearn.datasets import make_blobs

#--para utilizar las funciones de sklearn relacionadas con K-means
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

Generamos tres datasets con diferentes características con el objetivo de estudiar el desempeño de los algoritmos. Los dos primeros datasets tendran clusters bien definidos y el tercero tendra clusters que se entrelazan entre si.

In [ ]:
#--número de muestras a generar
num_muestras = 1000

#--2 clusters bien definidos
X_a, y_a = make_blobs(n_samples = 1000, n_features = 2, centers = [[3,3],[7,7]], random_state=0)

#--4 clusters bien definidos
X_b, y_b = make_blobs(n_samples = 1000, n_features = 2, centers = [[1,1],[7,7], [10,0], [0,12]],
                      random_state=0)

#--4 clusters con traslape
X_c, y_c = make_blobs(n_samples = 1000, n_features = 2, centers = 4, random_state=0)

#--ploteamos los datasets que vamos a utilizar
plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.scatter(X_a[:,0], X_a[:,1], marker='o', c = y_a)
plt.subplot(1,3,2)
plt.scatter(X_b[:,0], X_b[:,1], marker='o', c = y_b)
plt.subplot(1,3,3)
plt.scatter(X_c[:,0], X_c[:,1], marker='o', c = y_c)
plt.show()

Utilizamos el algoritmo "comun" de k-means (la versión que __no__ es online) desde la libería de sklearn para tener una referencia del desempeño.

Además, asumimos que sabemos el número de clusters, en la práctica podríamos utilizar el método del "codo" para inferir el número de clusters.

In [ ]:
#--primer dataset
model = KMeans(n_init=5, n_clusters = 2)
model.fit(X_a)

plt.figure(figsize=(15, 5))
plt.subplot(1,3,1)
plt.scatter(X_a[:,0], X_a[:,1], marker='o', c=model.labels_)

#--segundo dataset
model = KMeans(n_init=5, n_clusters = 4)
model.fit(X_b)
plt.subplot(1,3,2)
plt.scatter(X_b[:,0], X_b[:,1], marker='o', c=model.labels_)

#--tercer dataset
model = KMeans(n_init=5, n_clusters = 4)
model.fit(X_c)
plt.subplot(1,3,3)
plt.scatter(X_c[:,0], X_c[:,1], marker='o', c=model.labels_)
plt.show();

# K-means online

Utilizaremos los mismos datasets, estan cargados en memoria, sin embargo los entrenaremos usando solo algunas muestras a la vez, tal como lo haríamos si tuvieramos que cargar el dataset en memoria poco a poco.


## Usando solamente numpy

In [ ]:
# genero un indice aleatorio para permutar las filas y obtener diferentes
# respuestas cada vez que corro el algoritmo

idx = np.random.permutation(num_muestras)

#--actualizar estas variables dependiendo del data set que vayamos o ocupar.
num_clusters = 4
X = X_c[idx, :]
y = y_c[idx]


#--seleccionamos "num_clusters" muestras de forma aleatoria para inicializar
# los clusters. Asumimos que nuestro dataset no esta ordenado, entonces podemos
# utilizar las primeras muestras para inicializar los clusters.
#mu = X[np.random.randint(num_muestras, size=(num_clusters)), :].copy()
mu = X[:num_clusters, :].copy()
#--definimos una variable para llevar la cuenta de las muestras
n = np.zeros(num_clusters)
#--definimos una lista donde guardar la asignación de cada muestra a su cluster
y_hat = []
for i in range(num_muestras):
    #--distancia euclidiana de la muestra a los clusters
    d2 = np.sum((X[i,:] - mu)**2, axis=1)
    #--escoger el cluster mas cercano
    closest_cl = np.argmin(d2)
    #--actualizar las variables
    n[closest_cl] = n[closest_cl] + 1
    mu[closest_cl] = mu[closest_cl] + (1/n[closest_cl])*(X[i,:] - mu[closest_cl, :])
    #--guardamos el resultado
    y_hat.append(closest_cl)

#--prediccion
plt.scatter(X[:,0], X[:,1], marker='o', c=y_hat)
plt.scatter(mu[:,0], mu[:,1], marker='*', c = 'r', s=200)
plt.show()

## MiniBatchKMeans

Finalmente vamos a utilizar una función de sklearn que nos permite hacer el entrenamiento en línea.

In [ ]:
#--definimos el tamaño de cada mini-batch
mini_batch = 10

#--para el primer dataset
#--definición del modelo
modelo = MiniBatchKMeans(n_clusters = 2, random_state = 0, batch_size = mini_batch, n_init='auto')
#--entrenamiento
for i in range(mini_batch, num_muestras, mini_batch):
    X_batch = X_a[:i,]
    modelo.partial_fit(X_batch)
#--prediccion
y_hat = modelo.predict(X_a)
#--ploteo
plt.figure(figsize=(15, 5))
plt.subplot(1,3,1)
plt.scatter(X_a[:,0], X_a[:,1], marker='o', c=y_hat)

#--para el segundo dataset
#--definición del modelo
modelo = MiniBatchKMeans(n_clusters = 4, random_state = 0, batch_size = mini_batch, n_init='auto')
#--entrenamiento
for i in range(mini_batch, num_muestras, mini_batch):
    X_batch = X_b[:i,]
    modelo.partial_fit(X_batch)
#--prediccion
y_hat = modelo.predict(X_b)
#--ploteo
plt.subplot(1,3,2)
plt.scatter(X_b[:,0], X_b[:,1], marker='o', c=y_hat)

#--para el tercer dataset
#--definición del modelo
modelo = MiniBatchKMeans(n_clusters = 4, random_state = 0, batch_size = mini_batch, n_init='auto')
#--entrenamiento
for i in range(mini_batch, num_muestras, mini_batch):
    X_batch = X_c[:i,]
    modelo.partial_fit(X_batch)
#--prediccion
y_hat = modelo.predict(X_c)
#--ploteo
plt.subplot(1,3,3)
plt.scatter(X_c[:,0], X_c[:,1], marker='o', c=y_hat)
plt.show()